In [ ]:
arquivos o diretorio input - docker compose up - acesso container namenode e criação de estrutura diretório HDFS
e transferencia dos arquivos csv

hdfs dfs -mkdir -p /user/spark/projeto_final_basico

hdfs dfs -put /input/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv /user/spark/projeto_final_basico
hdfs dfs -put /input/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv /user/spark/projeto_final_basico


In [ ]:
import pyspark as spark
from pyspark.sql.functions import *

spark = SparkSession\
.builder\
.appName('Projeto final Básico - Campanha Nacional de Vacinação contra Covid-19')\
.config('spark.some.config.option', 'some-value')\
.enableHiveSupport()\
.getOrCreate()

#### Investiguei um dos arquivos CSV para entender como ele está formatado. Defini para esse projeto dois arquivos csv.
> O arquivo parte 1 encobre dados do primeiro semestre de 2021 e o arquivo parte 2 encobre dados dos primeiros 6 dias do mês de julho de 2021.

In [ ]:
!hdfs dfs -cat /user/leo-silva/Documentos/Semantix/spark/projeto_final/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv

In [ ]:
!hdfs dfs -cat /user/leo-silva/Documentos/Semantix/spark/projeto_final/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv

#### Identificado o formato do arquivo, criei o dataframe através do diretório do hdfs. Visualizei como está Schema e alterei o formato do campo data. (Início dos tratamento dos dados).

In [ ]:
csv_df = spark.read.csv('hdfs://namenode/user/spark/projeto_final_basico', sep=";",header=True, inferSchema=True, ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)
csv_df.show(10, vertical=True)

In [ ]:
csv_df.printSchema()

In [ ]:
csv_df_to_unix.show(30, vertical=True)

#### Criei o Banco de dados "covid" e particionei por Municipio ( como pede o exercício )

In [ ]:
spark.sql("create database covid")

In [ ]:
csv_df_to_unix.write.mode('overwrite').partitionBy('municipio').format('csv').saveAsTable('covid.municipio', path='hdfs://namenode:8020/user/hive/warehouse/covid_municipio/')

In [ ]:
!hdfs dfs -ls /user/hive/warehouse/covid_municipio

In [ ]:
spark.sql("SHOW DATABASES").show()

In [ ]:
spark.sql("USE covid")

In [ ]:
spark.sql("SHOW TABLES").show()

In [ ]:
spark.sql("SELECT * FROM municipio").show(200,vertical=True)

In [ ]:
# VISUALIZAÇÃO 1 | Obitos estados do maior para o menor -> 1º Semestre 2021 (01/01/2021 - 30/06/2021)
estado_obitos = spark.sql("SELECT estado, MAX(obitosAcumulado) AS obitos FROM municipio WHERE estado IS NOT NULL GROUP BY estado ORDER BY obitos DESC")
estado_obitos.show()

In [ ]:
# VISUALIZAÇÃO 1.1 | Óbitos pelas regiões do Brasil e Brasil como um todo. ( Brasil inicia o ano de 2021 com 195.411 óbitos )
regiao_br_obitos = spark.sql("select regiao, max(obitosAcumulado) as obitos from municipio group by regiao order by obitos desc")
regiao_br_obitos.show()

In [ ]:
# VISUALIZAÇÃO 2 | Número total de casos novos no fim do primeiro semestre de 2021.
casos_novos = spark.sql("SELECT estado, sum(casosNovos) AS casos_novos FROM municipio where estado IS NOT NULL group by estado order by casos_novos desc")
casos_novos.show()

In [ ]:
# Visualizaçaõ 3 | Valor médio de casos novos e óbitos diários no primeiro semestre por estado.
casos_obitos_media = spark.sql("SELECT estado, ROUND(SUM(casosNovos) / COUNT(data),2) AS media_casos_novos , ROUND(AVG(obitosAcumulado),2) AS media_obitos_diarios FROM municipio WHERE estado IS NOT NULL GROUP BY estado ORDER BY media_casos_novos DESC")
casos_obitos_media.show()

#### Após as 3 visualizações criadas, salvei a 1º como tabela HIVE

In [ ]:
estado_obitos.write.format('csv').saveAsTable('Obitos_por_estado')

In [ ]:
regiao_br_obitos.write.format('csv').saveAsTable('Obitos_por_regiao')

In [ ]:
#Visualizar as tabelas salvas 
spark.sql('SHOW TABLES').show()

#### A 2ª com formato parquet e compressão snappy

In [ ]:
casos_novos.write.option('compression', 'snappy').parquet('/user/spark/projeto_final_basico/segunda_visualizacao')

In [ ]:
#Conferindo se foi salvo corretamente
!hdfs dfs -ls '/user/spark/projeto_final_basico/segunda_visualizacao'

#### A 3ª em um tópico no Kafka

In [ ]:
casos_obitos_media.selectExpr("to_json(struct(*)) AS value").write.format('kafka').option('kafka.bootstrap.servers', 'kafka:9092').option('topic', 'casos_obitos_media').save()

In [ ]:
topic = spark.read.format('kafka').option('kafka.bootstrap.servers', 'kafka:9092').option('subscribe','casos_obitos_media').load()

topic_media_casos_obitos = topic.select(col('value').cast('string'))
topic_media_casos_obitos.show(truncate = False)

#### Agora criei uma visualização geral no Spark com todos os dados enviados para o HDFS : Síntese de casos, óbitos, incidência e mortalidade

##### Não consegui criar com SQL Querys

In [ ]:
df_spark_geral = spark.sql('SELECT regiao, estado, MAX(populacaoTCU2019) AS maximo_populacao_2019, MAX(casosAcumulado) AS maximo_casos_acumulados,  MAX(obitosAcumulado) AS maximo_obitos_acumulados FROM municipio GROUP BY  estado, regiao ORDER BY regiao ASC')

In [ ]:
df_spark_geral.show()

In [ ]:
df_spark_geral2 = spark.sql('SELECT regiao, estado, ROUND(MAX(casosAcumulado)/ MAX(populacaoTCU2019)*100000,0) AS incidencia, ROUND(MAX(obitosAcumulado)/ MAX(populacaoTCU2019)*100000,0) AS mortalidade FROM municipio GROUP BY  estado, regiao ORDER BY regiao ASC')

In [ ]:
df_spark_geral2.show()

In [ ]:
df_geral = csv_df.groupBy(['regiao', 'estado']).agg({'casosAcumulado':'max', 'obitosAcumulado':'max', 'populacaoTCU2019':'max'})

In [ ]:
df_geral.show(10)

In [ ]:
df_renomear_campos = df_geral.withColumnRenamed('max(populacaoTCU2019)','populacao').withColumnRenamed('max(casosAcumulado)', 'casos_acumulados').withColumnRenamed('max(obitosAcumulado)','obitos_acumulados')

In [ ]:
df_geral_completo = (df_renomear_campos.withColumn('incidencia', round(df_renomear_campos['casos_acumulados']/df_renomear_campos['populacao']*100000,1)).withColumn('mortalidade', round(df_renomear_campos['obitos_acumulados']/df_renomear_campos['populacao']*100000,1)))

In [ ]:
df_geral_completo.show(10)

#### Salvar a visualização do exercício 6 em um tópico no Elastic

In [ ]:
df_final = topic_media_casos_obitos
df_final.write.format("csv").save('hdfs://namenode/user/spark/projeto_final_basico/visualizacao3/covid_br.csv')

In [ ]:
!hdfs dfs -ls /user/spark/projeto_final_basico/visualizacao3/covid_br.csv

In [ ]:
!hdfs dfs -get /user/spark/projeto_final_basico/visualizacao3/covid_br.csv /input

##### POR ALGUM MOTIVO QUE DESCONHEÇO E DEPOIS DE MUITA PESQUISA E DIVERSAS TENTATIVAS NÃO COMPREENDI O MOTIVO DO PORQUE O ARQUIVO "covid_br.csv" NÃO FOI SALVO EM MINHA MÁQUINA LOCALMENTE..IMPOSSIBILITANDO-ME DE FINALIZAR O PROJETO..

>Professor Rodrigo espero que considere todo o projeto até aqui, realmente localmente o arquivo não existe mesmo constando essa mensagem ao dar run na célula "!hdfs dfs -get /user/spark/projeto_final_basico/visualizacao3/covid_br.csv /input"
>
>A mensagem que aparece é : 
>get: `/input/covid_br.csv/_SUCCESS': File exists`
>
>get: `/input/covid_br.csv/part-00000-a5470f49-ccb0-44fa-befd-7e89afd283f7-c000.csv': File exists`
>
>Porém não há nada no diretório..
>
>Obrigado.